In [12]:
#Libraries required 
from ddr.esutils import EsUtils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from mpl_toolkits import mplot3d
import getpass
import seaborn as sns

In [13]:
# Setup DDR ES Connection
host = 'https://search-dh-tti-digital-biomarker-es-5hrwjtje3nybikyzmyx3rcx2qm.us-east-2.es.amazonaws.com/'
username = getpass.getpass("Enter TT&I Data Portal Username: ")
password = getpass.getpass("Enter TT&I Data Portal Password: ")
eu = EsUtils(host, username, password)
metadata = eu.get_metadata()
#del username
#del password

Enter TT&I Data Portal Username: ········
Enter TT&I Data Portal Password: ········


In [14]:
# creating EsUtils object
eu = EsUtils(host, username, password)

In [32]:
# getting the metadata
metadata = eu.get_metadata()

In [35]:
metadata.add_study('cpmp')

In [36]:
metadata.add_timeCols(indx='cpmp_bp02_hourly_compliance', localCol = 'local_date')

metadata.refresh_metadata()

In [37]:
# get the list of studies
metadata.list_studies()

,study
0,fusion
1,hbeh
2,pacap
3,uma
4,atd
5,covid19
6,o016
7,eiporta
8,bari
9,platform_demo


####  Exploring cpmp_p2x7 variables/indexs

In [38]:
index_col=metadata.list_variables("cpmp")
index_col

,index
0,cpmp_bp02_hourly_compliance
1,cpmp_bp03_compliant_visits
2,cpmp_bp03_compliant_visits_pa
3,cpmp_bp03_compliant_visits_sleep
4,cpmp_bp05_hourly_compliance
5,cpmp_device_daily_complianc_heatmap
6,cpmp_device_daily_compliance_heatmap_20hrs_66
7,cpmp_device_daily_compliance_v2_20hrs_66
8,cpmp_device_site_average_compliance_20hrs
9,cpmp_device_site_average_compliance_20hrs_66


#### Understaing the various fields in cpmp_p2x7 

In [39]:
d = {}

for indx in index_col['index']:
    print(indx)
    d[indx] =metadata.list_fields(indx)
    print(d[indx])
    

cpmp_bp02_hourly_compliance
               field   type
0      compliant_min   long
1               hour   long
2         local_date   date
3  not_recording_min   long
4            site_id   text
5            subject   text
6          trial_day  float
cpmp_bp03_compliant_visits
            field  type
0  compliant_days  long
1         subject  text
2           visit  text
cpmp_bp03_compliant_visits_pa
            field  type
0  compliant_days  long
1         subject  text
2           visit  text
cpmp_bp03_compliant_visits_sleep
            field  type
0  compliant_days  long
1         subject  text
2           visit  text
cpmp_bp05_hourly_compliance
               field  type
0      compliant_min  long
1               hour  long
2         local_date  date
3  not_recording_min  long
4            site_id  text
5            subject  text
6          trial_day  long
7              visit  text
cpmp_device_daily_complianc_heatmap
cpmp_device_daily_complianc_heatmap is not a valid index. Use l

                            field   type
0                  compliant_days   long
1   days_data_received_percentage  float
2       device_swaps_during_visit   long
3          expected_days_in_visit   long
4                             isa   text
5         receieved_days_in_visit   long
6                            site   long
7                         subject   text
8   timezone_changes_during_visit   long
9                           visit   text
10    visit_compliance_percentage  float
cpmp_weekly_compliance_20hrs_66_baseline_8weeks
        field   type
0    baseline  float
1         isa   text
2   startdate   date
3     subject   text
4       week1  float
5       week2  float
6       week3  float
7       week4  float
8       week5  float
9       week6  float
10      week7  float
11      week8  float
cpmp_weekly_compliance_heatmap_20hrs_66_baseline_8weeks
            field  type
0  compliant_days  long
1             isa  text
2       startdate  date
3         subject  text
4          

In [40]:
# Define your query params
compliance_data_index_list =['cpmp_bp02_hourly_compliance']

df = None
for index in compliance_data_index_list:
    if df is None:
        df = eu.query(indx=index)
    else:
        df = pd.concat([df, eu.query(indx=index)], ignore_index=True)


df

,local_date,hour,subject,site_id,compliant_min,not_recording_min,trial_day
0,2022-03-21,14,13368,102,0,60,1.0
1,2022-03-21,2,13368,102,0,60,1.0
2,2022-03-21,1,13368,102,0,60,1.0
3,2022-03-21,8,13368,102,0,60,1.0
4,2022-03-21,0,13368,102,0,60,1.0
...,...,...,...,...,...,...,...
170995,2022-05-02,17,13415,142,60,0,36.0
170996,2022-05-03,1,13415,142,0,0,37.0
170997,2022-05-03,2,13415,142,0,0,37.0
170998,2022-05-03,7,13415,142,0,59,37.0


In [45]:
df2 = df[df["subject"].isin(["13058"])]

In [46]:
df2

,local_date,hour,subject,site_id,compliant_min,not_recording_min,trial_day


In [41]:
df1 = df[df["subject"].isin(["13038"])]
df2 = df[df["subject"].isin(["13058"])]
df3 = df[df["subject"].isin(["13536"])]

In [50]:
df1.to_csv('cpmp_13038.csv')

In [51]:
df3.to_csv('cpmp_13536.csv')

In [47]:
df2 = df[df["subject"].isin(["13058"])]
df2

,local_date,hour,subject,site_id,compliant_min,not_recording_min,trial_day


In [16]:
df2.to_csv('C:/Users/shantam.bhuraria@lilly.com/Desktop/cpmp_hourly_compliance_files/14356.csv')